# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
import random

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'train.p'
validation_file= 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train_origin, y_train_origin = train['features'], train['labels']

X_train, X_valid, y_train, y_valid = train_test_split(X_train_origin, y_train_origin, test_size=0.2, random_state=42)
#X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [2]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

#
n_valid = X_valid.shape[0]
# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(list(set(train['labels'])))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_valid)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 27839
Number of validation examples = 6960
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


In [3]:
## Split Training data to training and validation

In [4]:
import collections
x_test = X_train
y_test_label = y_train

tuple_list = []  
for i in range(x_test.shape[0]):
    tuple_list.append( (x_test[i], y_test_label[i]) )  
sort_data = sorted(tuple_list, key=lambda x: x[1]) #list of tuple

#print (sort_data[179][1]) #0 
#print (sort_data[180][1]) #1
#print (sort_data[180+1980-1][1]) #1
#print (sort_data[180+1980][1]) #2
 
    
#a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
# print number of images for each class

counter=collections.Counter(y_train)
print(counter)

# create index range 
class_index = [None] * n_classes
for i in range(n_classes): 
    if i == 0:        
        class_index[i] = (0 , counter[i]-1)
    else:
        class_index[i] = (class_index[i-1][1]+1 ,class_index[i-1][1]+counter[i])

print ('class_index_range: ', class_index)
# sort class according to their labels
# check if range okay
#for i in range(len(class_index)):
#    print ( sort_data [class_index[i][0]] [1], sort_data[class_index[i][1]][1] )


Counter({2: 1616, 1: 1555, 13: 1534, 38: 1490, 12: 1473, 10: 1468, 4: 1397, 5: 1312, 25: 1089, 9: 1067, 7: 1048, 8: 1023, 3: 1018, 11: 941, 18: 861, 35: 840, 17: 787, 14: 560, 31: 551, 33: 491, 26: 430, 15: 415, 28: 382, 23: 373, 6: 304, 30: 301, 34: 286, 16: 283, 22: 265, 36: 263, 20: 243, 40: 230, 39: 217, 21: 209, 29: 200, 24: 198, 41: 182, 32: 174, 42: 166, 27: 158, 19: 154, 37: 143, 0: 142})
class_index_range:  [(0, 141), (142, 1696), (1697, 3312), (3313, 4330), (4331, 5727), (5728, 7039), (7040, 7343), (7344, 8391), (8392, 9414), (9415, 10481), (10482, 11949), (11950, 12890), (12891, 14363), (14364, 15897), (15898, 16457), (16458, 16872), (16873, 17155), (17156, 17942), (17943, 18803), (18804, 18957), (18958, 19200), (19201, 19409), (19410, 19674), (19675, 20047), (20048, 20245), (20246, 21334), (21335, 21764), (21765, 21922), (21923, 22304), (22305, 22504), (22505, 22805), (22806, 23356), (23357, 23530), (23531, 24021), (24022, 24307), (24308, 25147), (25148, 25410), (25411, 255

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
# Visualizations will be shown in the notebook.
%matplotlib inline

# sort and do statistics the training data set 
import collections
x_test = X_train 
y_test_label = y_train
tuple_list = []  
for i in range(x_test.shape[0]):
    tuple_list.append( (x_test[i], y_test_label[i]) )  
sort_data = sorted(tuple_list, key=lambda x: x[1]) #list of tuple

counter=collections.Counter(y_train)
# print the number of images for each class
print(counter)


# create index range 
#class_index = [None] * n_classes
#for i in range(n_classes): 
#    if i == 0:        
#        class_index[i] = (0 , counter[i]-1)
#    else:
#        class_index[i] = (class_index[i-1][1]+1 ,class_index[i-1][1]+counter[i])


# random plot 10 figure for each class
#number_image_column = 10
#fig = plt.figure(1, (50, 50))
#grid = ImageGrid(fig, 111,  # similar to subplot(111)
#                 nrows_ncols=(43, 10),  # creates 2x2 grid of axes
#                 axes_pad=0.1,  # pad between axes in inch.
#                 )
# visualization
#for i in range(43):
#    for j in range(number_image_column):
#        index = random.randint(class_index[i][0], class_index[i][1])   
#        im = sort_data[index][0]    
#        grid[i*number_image_column+j].imshow(im) 


In [ ]:
def visulize_random_training_data(X_train, y_train):
    # sort data
    tuple_list = []  
    for i in range(X_train.shape[0]):
        tuple_list.append( (X_train[i], y_train[i]) )
        sort_data = sorted(tuple_list, key=lambda x: x[1]) #list of tuple
    
    counter=collections.Counter(y_train)

    class_index = [None] * n_classes
    for i in range(n_classes): 
        if i == 0:        
            class_index[i] = (0 , counter[i]-1)
        else:
            class_index[i] = (class_index[i-1][1]+1 ,class_index[i-1][1]+counter[i])

    
    
    number_image_column = 10
    fig = plt.figure(1, (50, 50))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(43, 10),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    
    
    # visualization
    for i in range(43):
        for j in range(number_image_column):
            index = random.randint(class_index[i][0], class_index[i][1])   
            im = sort_data[index][0]    
            grid[i*number_image_column+j].imshow(im) 

    

In [ ]:
visulize_random_training_data(X_train, y_train)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Pre-process the Data Set (normalization, grayscale, etc.)

In [5]:
X_train.dtype

dtype('uint8')

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

### Balencing training set

In [6]:
# create a new training set with equal number of class
class_index
number_pro_class = 2000
new_X_train = np.zeros([number_pro_class * n_classes , 32,32,3], dtype=np.uint8)
new_y_train = np.zeros(number_pro_class * n_classes)

print (new_X_train.shape)


iterator = 0
for i in range(43):
    for j in range(number_pro_class):
        index = random.randint(class_index[i][0], class_index[i][1])          
        new_X_train[iterator] = sort_data[index][0]
        new_y_train[iterator] = sort_data[index][1]
        iterator +=1

print (new_X_train.shape)
print (new_y_train.shape)

# visulize_random_training_data(new_X_train, new_y_train)
# randomlize 
X_train, X_valid_tmp, y_train, y_valid_tmp = train_test_split(new_X_train, new_y_train, test_size=0.0, random_state=42)




(86000, 32, 32, 3)
(86000, 32, 32, 3)
(86000,)


In [7]:
def preprocessing(X):
    # normalization 
    x_norm = X/255.0
    return x_norm

In [8]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.
import numpy as np
# normalize a image
X_train =  preprocessing(X_train)
X_valid = preprocessing(X_valid)

### Model Architecture

In [9]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
from tensorflow.contrib.layers import flatten
import tensorflow as tf
def LeNetSolution(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, n_classes), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(n_classes))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    weights = {
    'wc1': tf.Variable(tf.truncated_normal([5, 5, 3, 6],   mean = mu, stddev = sigma)),
    'wc2': tf.Variable(tf.truncated_normal([5, 5, 6, 16],  mean = mu, stddev = sigma)),
    'wc3': tf.Variable(tf.truncated_normal([400,120],      mean = mu, stddev = sigma)),
    'wc4': tf.Variable(tf.truncated_normal([120,84],       mean = mu, stddev = sigma)),
    'wc5': tf.Variable(tf.truncated_normal([84,n_classes], mean = mu, stddev = sigma))
        
    }
    
    bias = {
    'b1':  tf.Variable(tf.zeros(6)),
    'b2':  tf.Variable(tf.zeros(16)),
    'b3':  tf.Variable(tf.zeros(120)),
    'b4':  tf.Variable(tf.zeros(84)),
    'b5':  tf.Variable(tf.zeros(n_classes))

    }

    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv = tf.nn.conv2d(x, weights['wc1'], [1,1,1,1], 'VALID') + bias['b1']
    
    # TODO: Activation.
    conv = tf.nn.relu(conv)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv = tf.nn.conv2d(conv, weights['wc2'], [1,1,1,1], 'VALID') + bias['b2']

    # TODO: Activation.
    conv = tf.nn.relu(conv)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    conv = flatten(conv)

   
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    conv = tf.add(tf.matmul(conv, weights['wc3']), bias['b3'])
    
    # TODO: Activation.
    conv = tf.nn.relu(conv)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    conv = tf.add(tf.matmul(conv, weights['wc4']), bias['b4'])
    
    # TODO: Activation.
    conv = tf.nn.relu(conv)
    
    # Test add drop out 
    conv = tf.nn.dropout(conv , keep_prob)

    
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    logits = tf.add(tf.matmul(conv, weights['wc5']), bias['b5'])

    return logits


## Test output dimension

In [10]:
test_out = LeNet(X_train[0:3])
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    output = session.run( test_out  )

    print (output.shape)

TypeError: Input 'filter' of 'Conv2D' Op has type float32 that does not match type float64 of argument 'input'.

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [11]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)

one_hot_y = tf.one_hot(y, n_classes)
EPOCHS = 20
BATCH_SIZE = 128

rate = 0.001
logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

# shuffle training data 
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)



In [12]:
y_train.shape

(86000,)

## Model evaluation

In [13]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    total_loss = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y , keep_prob:1.0})        
        total_accuracy += (accuracy * len(batch_x))       
        
    return total_accuracy / num_examples  

In [15]:
## Training

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob:0.5})
            #loss = sess.run(loss_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        validation_accuracy_train = evaluate(X_train, y_train)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Training Accuracy = {:.3f}".format(validation_accuracy_train))
        
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.752
Training Accuracy = 0.828

EPOCH 2 ...
Validation Accuracy = 0.816
Training Accuracy = 0.890

EPOCH 3 ...
Validation Accuracy = 0.900
Training Accuracy = 0.943

EPOCH 4 ...
Validation Accuracy = 0.906
Training Accuracy = 0.952

EPOCH 5 ...
Validation Accuracy = 0.928
Training Accuracy = 0.971

EPOCH 6 ...
Validation Accuracy = 0.945
Training Accuracy = 0.982

EPOCH 7 ...
Validation Accuracy = 0.945
Training Accuracy = 0.983

EPOCH 8 ...
Validation Accuracy = 0.950
Training Accuracy = 0.984

EPOCH 9 ...
Validation Accuracy = 0.958
Training Accuracy = 0.989

EPOCH 10 ...
Validation Accuracy = 0.958
Training Accuracy = 0.990

EPOCH 11 ...
Validation Accuracy = 0.951
Training Accuracy = 0.986

EPOCH 12 ...
Validation Accuracy = 0.968
Training Accuracy = 0.995

EPOCH 13 ...
Validation Accuracy = 0.973
Training Accuracy = 0.996

EPOCH 14 ...
Validation Accuracy = 0.962
Training Accuracy = 0.992

EPOCH 15 ...
Validation Accuracy = 0.965
Tra

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 